# Rasterize Natural Gas Pipeline Network
Creates a raster version at the same extent and cell size of the MIT cost surface.

In [1]:
import rasterio
from rasterio import features
import geopandas as gpd
import numpy as np

In [3]:
#Load the cost surface raster
cost_surface_dataset = rasterio.open('../data/MIT_Surface_Full/costsurface500m.img')
#Get the surface 
cost_surface = cost_surface_dataset.read(1)
#Extract the coordinate reference system
cost_crs = cost_surface_dataset.crs
#Get the metadata
cost_meta = cost_surface_dataset.meta

In [4]:
#Read in features
pipelines_gdb = 'C:\\Workspace\\Rextag_7May2020\\Rextag_Zone2_Natural_Gas.gdb'
pipelines_lyr = 'NaturalGasPipelines'

In [5]:
#Read in pipeline dataset as a geodataframe
gdf = gpd.read_file(pipelines_gdb,driver='FileGDB',layer=pipelines_lyr)

In [8]:
gdf.shape

(630204, 42)

In [9]:
#Extract only operational pipeline routes
gdf_operational = gdf.loc[gdf['STATUS'] == 'Operational'].reset_index()
gdf_operational.shape

(622065, 43)

In [10]:
#Transform to same crs as cost surface
gdf_operational = gdf_operational.to_crs(cost_crs)

In [12]:
#Convert features to raster
image = features.rasterize(shapes=gdf_operational.geometry,
                           out_shape=cost_surface_dataset.shape,
                           transform=cost_surface_dataset.transform,
                           all_touched=False)

In [13]:
#Update metadata for output
cost_meta.update(
    {'driver':'GTiff',
     'dtype':'uint8',
     'nodata':0,
    })

In [14]:
#Export as tiff
with rasterio.open('..\\data\\processed\\USA_pipes_operational.tif','w',**cost_meta) as dst:
    dst.write(image, indexes=1)